In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from uszipcode import SearchEngine

In [2]:
job_title = "astronaut"
url = f'https://www.indeed.com/jobs?q={job_title}&limit=50'

In [3]:
def scrapeData(class_str, list_name):
    element = soup.find_all(class_=class_str)
    for i in element:
        list_name.append(i.text.strip())

In [4]:
def scrapeSalary(num, list_name):
    salary_tags = soup.find_all(class_='row')
    for i in range(len(salary_tags)):
        try:
            salary = salary_tags[i].find(class_='salary').text.strip()
            list_name.append(salary)
        except:
            list_name.append("No Salary Info Provided")

In [5]:
def scrapeLink(list_name):
    frontend = 'https://www.indeed.com/'
    links = soup.find_all('div', class_='title')
    for link in links:
        backend = link.find('a')['href']
        list_name.append(frontend+backend)

In [6]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [7]:
job_titles = []
company_names = []
locations = []
salaries = []
link_list = []

In [8]:
scrapeData('title', job_titles)
scrapeData('company', company_names)
scrapeData('location', locations)
num = len(job_titles)
scrapeSalary(num, salaries)
scrapeLink(link_list)

In [9]:
url2 = f'https://www.indeed.com/jobs?q={job_title}&limit=50'
browser.visit(url2)
html = browser.html
soup2 = bs(html, 'html.parser')
browser.quit()

In [10]:
scrapeData('title', job_titles)
scrapeData('company', company_names)
scrapeData('location', locations)
num = len(job_titles)
scrapeSalary(num, salaries)
scrapeLink(link_list)

In [11]:
def hasZip(inputString):
    if any(char.isdigit() for char in inputString):
        #return inputString[-5:]
        return "".join(filter(lambda x: x.isdigit(), inputString))
    else:
        return inputString

In [12]:
loc_list = []
for i in locations:
   loc_list.append(hasZip(i))

In [13]:
def getZip(str, zipList):
    if str.isdigit():
        zipList.append(str)
    else:
        try:
            search = SearchEngine(simple_zipcode=True)
            city_state_str = str.split(', ')
            city_state = search.by_city_and_state(city_state_str[0], city_state_str[1])
            zipList.append(city_state[0].zipcode)
        except:
            zipList.append("None")
            print(f"No City, State info found for {str}")

In [14]:
zip_list = []

for i in loc_list:
    getZip(i, zip_list)

No City, State info found for Medley, FL
No City, State info found for Medley, FL
No City, State info found for Moffett Field, CA
No City, State info found for Medley, FL
No City, State info found for Medley, FL
No City, State info found for Moffett Field, CA


In [15]:
def getCoord(str, coordList, cityList):
    search = SearchEngine(simple_zipcode=True)
    if str.isdigit():
        try:
            zipcode = search.by_zipcode(str)
            coords = [zipcode.lat, zipcode.lng]
            coordList.append(coords)
            cityList.append(zipcode.post_office_city)
        except:
            coordList.append(["None", "None"])
            cityList.append("None")
            print(f"No Results Found for Zip {str}")
    else:
        coordList.append(["None", "None"])
        cityList.append(str)

In [16]:
search_limit = 100

In [17]:
coord_list = []
city_list = []

for i in zip_list[0:search_limit]:
    getCoord(i, coord_list, city_list)

In [18]:
job_list = []

for i in range(search_limit):
    job_dict = {}
    job_dict['Title'] = job_titles[i]
    job_dict['Company'] = company_names[i]
    job_dict['Location'] = city_list[i]
    job_dict["Coordinates"] = coord_list[i]
    job_dict["Salary_Info"] = salaries[i]
    job_dict['Link'] = link_list[i]
    job_list.append(job_dict)

In [19]:
for i in job_list:
    #print(i['Coordinates'])
    if "None" in str(i["Coordinates"]):
        job_list.remove(i)

In [20]:
len(job_list)

94

In [21]:
len(coord_list)

100

In [22]:
job_list

[{'Title': 'Help Wanted',
  'Company': 'Kinokuniya Los Angeles',
  'Location': 'Los Angeles, CA',
  'Coordinates': [33.97, -118.25],
  'Salary_Info': 'No Salary Info Provided',
  'Link': 'https://www.indeed.com//rc/clk?jk=56f98972f37b3f07&fccid=85f0a68c99df2faf&vjs=3'},
 {'Title': 'Summer Camp SCUBA Instructor',
  'Company': 'Guided Discoveries - Astrocamp',
  'Location': 'Idyllwild, CA',
  'Coordinates': [33.74, -116.72],
  'Salary_Info': '$400 a week',
  'Link': 'https://www.indeed.com//pagead/clk?mo=r&ad=-6NYlbfkN0DotyRzA45OhEOnnld2FgBxS7aeoan5svkpBu1hrOsFo8GAzlgrnZopLQ6pYAcY3UdOS32Ouk2Cf-KOm2kLt5_uTwxpf90utpwWCSPyHkSg4BSP-PSNS-UCTPBDNVNOk1IkDaDltrQnWyv2O7t3m5h_RsJc1-phHLUOWjiKEU1g500nr5Ovrie6eWeI-kmahcIgqGm8yYuXrU5QP8Xf5-vHPMHHczzKnBkn-kYPo0nVPJnuRUZEy2GRuudArgiFN0PeevAG_4c3vMF2LyVcYdTImMJl3-L8J8Aq3pFb_FiibE6_8fmbep3_2JkpurkXUrWfTRJSsJb7DK6ji5tvQuxYdIoWSh3AW_tkcN9HnB51JDqqGitmG9UHJAVSnJVuXdzBZn4FsVBqiUrOXkrHtrTLLCAs4vX5Yc9whjW9nLoJ-Iyo2o9zJA0T&p=0&fvj=1&vjs=3'},
 {'Title': 'Tech, Q

---

# MongoDB Integration with PyMongo

In [23]:
from pymongo import MongoClient

In [24]:
conn = 'mongodb://localhost:27017'
client = MongoClient(conn)
db = client['job_search_db']

db.search_results.drop()

col = db['search_results']

for x in job_list:
    col.insert_one(x)